In [66]:
import pandas as pd
import numpy as np
from scipy.signal import correlate
from tqdm import tqdm

from scipy.stats import pearsonr

In [61]:
def calculate_correlation(poly_division, time_interval, max_lags = 5):
    df = pd.read_csv(f"data/polygon_data/{poly_division}_{time_interval}.csv")
    signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
    
    features = df.columns.tolist()[2:]
    dist_matrix = np.zeros((len(features), len(features)))

    for i, sample in tqdm(enumerate(signal_multi)):
        dist_matrix_ = np.zeros((len(features), len(features)))
        for m in range(len(features)):
            sample[:, m] -= sample[:, m].mean()
            sample[:, m] /= sample[:, m].std() + 1e-6
            for n in range(len(features)):
                if m == n:
                    continue
                # calculate cross-correlation

                
                sample[:, n] -= sample[:, n].mean()
                sample[:, n] /= sample[:, n].std() + 1e-6
                xcorr = correlate(sample[:, m], sample[:, n], mode='full')
                xcorr = max(xcorr, key=abs)
                dist_matrix_[m, n] = xcorr
                # normalize cross-correlation
                div = np.sqrt(np.sum(sample[:, m]**2) * np.sum(sample[:, n]**2))
                div = div if div != 0 else 1
                dist_matrix_[m, n] /= div

        dist_matrix += dist_matrix_

    dist_matrix /= len(signal_multi)

    
    result = [
        
    ]

    for i, feat_i in enumerate(features):
        for j, feat_j in enumerate(features):
            if feat_i == feat_j:
                continue
            

            result.append({
                "row": feat_i,
                "column": feat_j,
                "value": dist_matrix[i, j],
                "lag": -777
            })

            

    result = pd.DataFrame(result)
    result.to_csv(f"data/similarity_matrix/{poly_division}_{time_interval}.csv", index=False)

In [96]:
np.arctanh(0)

0.0

In [97]:
def calculate_correlation(poly_division, time_interval, max_lags = 5):
    df = pd.read_csv(f"data/polygon_data/{poly_division}_{time_interval}.csv")
    signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
    
    features = df.columns.tolist()[2:]
    dist_matrix = np.zeros((len(features), len(features)))

    for i, sample in tqdm(enumerate(signal_multi)):
        dist_matrix_ = np.zeros((len(features), len(features)))
        for m in range(len(features)):
            if np.sum(sample[:, m]) == 0:
                continue
            for n in range(len(features)):
                if m == n:
                    continue
                if np.sum(sample[:, n]) == 0:
                    continue
                # calculate correlation
                
                xcorr = pearsonr(sample[:, m], sample[:, n])[0]
                if xcorr != 1:
                    xcorr = np.arctanh(xcorr)
                dist_matrix_[m, n] = xcorr

        
        dist_matrix += dist_matrix_

        dist_matrix__ = dist_matrix / (i + 1)
        dist_matrix__ = np.tanh(dist_matrix__)
        if dist_matrix__.max() >= 1:
            print(i, dist_matrix_, dist_matrix)
            return

    dist_matrix /= len(signal_multi)
    dist_matrix[dist_matrix > 0] = np.tanh(dist_matrix[dist_matrix > 0])
    
    result = [
        
    ]

    for i, feat_i in enumerate(features):
        for j, feat_j in enumerate(features):
            if feat_i == feat_j:
                continue
            

            result.append({
                "row": feat_i,
                "column": feat_j,
                "value": dist_matrix[i, j],
                "lag": -777
            })

            

    result = pd.DataFrame(result)
    result.to_csv(f"data/similarity_matrix/{poly_division}_{time_interval}.csv", index=False)

In [63]:
def calculate_correlation(poly_division, time_interval, max_lags = 5):
    df = pd.read_csv(f"data/polygon_data/{poly_division}_{time_interval}.csv")
    
    # create lagged columns
    df_new = df.copy()
    for lag in range(-max_lags, max_lags + 1):
        df_new = pd.merge(
            df_new,
            df.groupby("id_poly").apply(lambda x : x.set_index("date").shift(lag), include_groups=False).reset_index(),
            on=["id_poly", "date"],
            how = "inner",
            suffixes = ("", f"_lag_{lag}")
        )

    df_new.to_csv(f"data/polygon_data/{poly_division}_{time_interval}_lagged.csv", index=False)
    corr_matrix = df_new.iloc[:, 2:].corr()
    features = df.columns.tolist()[2:]

    result = []

    for i, feat_i in enumerate(features):
        correlations = corr_matrix[feat_i].to_dict()
        for j, feat_j in enumerate(features):
            if feat_i == feat_j:
                continue
            

            correlations_values = [correlations[feat_j + f"_lag_{lag}"] for lag in range(-max_lags, max_lags + 1)]

            # get lag and value of maximum absolute correlation
            max_corr = max(correlations_values, key=abs)
            max_corr_lag = correlations_values.index(max_corr) - max_lags

            result.append({
                "row": feat_i,
                "column": feat_j,
                "value": max_corr,
                "lag": max_corr_lag
            })

            

    result = pd.DataFrame(result)
    result.to_csv(f"data/similarity_matrix/{poly_division}_{time_interval}.csv", index=False)

In [98]:
calculate_correlation("SpCenterCensus5k", "Period2", 6)

/tmp/ipykernel_60107/1202181906.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
5000it [00:15, 329.60it/s]


In [80]:
calculate_correlation("BLACities", "Year2", 6)

/tmp/ipykernel_60107/2221237547.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
10it [00:00, 97.11it/s]

-2.8248507961926705 0.2714889405570338
-2.809536052317792 0.26267547419529325
-1.969675918405181 0.41983066192840446
-3.8250943111014437 0.4024761724862907
-0.8852758970087586 0.20492009580820883
-2.4298583669246154 0.7287689765006452
-1.9993750129210275 0.8873090334883458
-3.27765356673444 0.20989235948917284
-3.119833081047339 0.3048742330045828
-1.6527949483481565 0.5826939219457832
-4.854537311580718 0.4025338177963728
-3.8820223548743082 0.44138336952567003
-3.6023539567697824 0.44607270916653174
-3.8094503233390316 0.22524637670586714
-4.05451334297241 0.4655876060255516
-2.483885956045293 0.6928746157573646
-2.400284677992117 0.6823876210993902
-3.2729309108463864 0.21426647762398576
-1.8201149646264394 0.35902913455487007


29it [00:00, 85.36it/s]

-2.256199983196048 0.40449777809579995
-3.3911423248285284 0.8814896307714168
-1.2871976449659674 0.41286173484455224
-2.95040559340487 0.5196417939505988
-4.766230607112085 0.22129232744543423
-4.163183001833021 0.36124138876326256
-3.334344097874368 0.4544764448962254
-2.33213905993249 0.41532990788288376
-5.672550076714888 0.14829285243531487
-2.9919005151268228 0.27371650623108634
-4.036965514917954 0.45989853528087843
-4.384664771402795 0.33134676709865063
-1.4253198706061743 0.30833870581579464
-2.393369693997887 0.347891528864326
-2.3236468427356063 0.7115588319183097
-2.3442199177039464 0.31033548754810353
-3.717378592331159 0.4515547114377682


47it [00:00, 84.56it/s]

...

-1.436933538712606 0.6492934319614766
-1.4703844613345654 0.31553317693293265
-0.37375747964064715 0.2957161170951158
-2.4848042976535196 0.3096580327391099
-0.836701965537278 0.36369594992830817
-0.7932643377036751 0.3135084843730695
-1.208173843195719 0.47165921994249793
-0.4726826621295138 0.24742721413865087
-1.2077878934064197 0.39865427099504286
-0.47333880730772304 0.4128840423720019
-0.8890714207954331 0.8316594542551486
-1.2889805518472386 0.5858630532063293


767it [00:09, 58.12it/s]

-1.3672168661405406 0.3294182998986468
-0.8056978424443574 0.7275688915556414
-2.43323449611794 0.7820119150636242
-1.0293045507539336 0.27221368123941714
-0.7925776829942679 0.5475091586263198
-1.1576698397950234 0.20929480150493804
-2.487021645250212 0.33194705824891557
-0.6119620709827607 0.5327126587893991
-0.5378890801452374 0.25523939629279646
-2.705223647644052 0.9305089229947653
-2.203483771249096 0.5314390531504
-2.2298259504421623 0.3329208850513479
-1.0273897015069993 0.5142270339224561


773it [00:09, 78.93it/s]

-4.194165325709037 0.2408905634724213


In [4]:
calculate_correlation("NYBlocks", "Period1", 10)

In [84]:
np.tanh(10)

0.9999999958776927